#### Lab 7 
#### Rouge Metrics

In [2]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import numpy as np
from fuzzywuzzy import fuzz 
import ipywidgets as widgets
import pprint
from ipywidgets import interact, interact_manual
import re
__PATH__ = "./data.csv"
df = pd.read_csv(__PATH__,sep=";",header=0)

#### Preprocessing the title to list of tokens

In [8]:
titles = list(df['title'].apply(
    lambda t : 
        tuple(
            filter(lambda e:not e in stopwords.words('english'),
                map(lambda e:e.lower(),
                       re.findall('([A-Z]{1}[a-z]+)',t.replace('.pdf','')))
                )
            )
        )
    )

res = {}
for title in titles[:20]:
    synsets = {}
    for word in title:
        synsets[word]=[synset for synset in wn.synsets(word)]
    res[title] = synsets
print(res)

{('much', 'card', 'deck', 'well', 'shuffled'): {'much': [Synset('much.n.01'), Synset('much.a.01'), Synset('much.r.01'), Synset('much.r.02'), Synset('a_lot.r.01'), Synset('much.r.04'), Synset('much.r.05')], 'card': [Synset('card.n.01'), Synset('card.n.02'), Synset('card.n.03'), Synset('card.n.04'), Synset('wag.n.01'), Synset('poster.n.01'), Synset('calling_card.n.02'), Synset('card.n.08'), Synset('menu.n.01'), Synset('batting_order.n.01'), Synset('circuit_board.n.01'), Synset('tease.v.07'), Synset('card.v.02')], 'deck': [Synset('deck.n.01'), Synset('deck.n.02'), Synset('pack_of_cards.n.01'), Synset('deck.n.04'), Synset('deck.v.01'), Synset('deck.v.02'), Synset('deck.v.03')], 'well': [Synset('well.n.01'), Synset('well.n.02'), Synset('well.n.03'), Synset('well.n.04'), Synset('well.n.05'), Synset('well.v.01'), Synset('well.a.01'), Synset('good.s.13'), Synset('well.s.03'), Synset('well.r.01'), Synset('well.r.02'), Synset('well.r.03'), Synset('well.r.04'), Synset('well.r.05'), Synset('well.r

In [9]:
def GetHprnms2(snst):
    hprnms1 = set()
    hprnms2 = set()
    lvl_cnt = 1
    while lvl_cnt < 3:
        snst = snst.hypernyms()[0]
#         print('snst = snst.hypernyms()[0]', snst)
        if(lvl_cnt == 1):
            hprnms1.add(snst)
#             print('hprnms1.add(snst)', snst)
        elif(lvl_cnt == 2):
            hprnms2.add(snst)
#             print('hprnms2.add(snst)', snst,'\n')
        lvl_cnt += 1
    return hprnms1, hprnms2

def ExtractHprnmsEng(a):
    hprnms1_eng = set()
    hprnms2_eng = set()
    for wrd in a:
        try:
            wrd_snst = wn.synsets(wrd)[0]
            hrnms_buff = GetHprnms2(wrd_snst)
#             print('hrnms_buff', hrnms_buff)
#             print('hrnms_buff[0]', hrnms_buff[0])
#             print('hrnms_buff[1]', hrnms_buff[1])
            hprnms1_eng = hrnms_buff[0]
#             print('hprnms1_eng', hprnms1_eng)
            hprnms2_eng = hrnms_buff[1]
#             print('hprnms2_eng', hprnms2_eng, '\n')
        except IndexError:
            continue
    return hprnms1_eng, hprnms2_eng

def F1Measure(a, b):
    a = set(a)
    b = set(b)
    overlap = set()
    overlap_hyp_cnt = 0
    overlap = a.intersection(b)
    overlap_hyp_cnt = len(overlap) 
    recall = overlap_hyp_cnt/len(a)
    precision = overlap_hyp_cnt/len(b)
    if len(overlap) == 0:
        return 0, overlap
    else:
        return 2*recall*precision/(recall+precision), overlap
    
lang = 'eng'

def distance(a, b):
    overlap_hyp_cnt = 0
    a = set(a) 
    b = set(b) 
    f1score, overlap = F1Measure(a, b)
    a = a - overlap
    b = b - overlap
    if len(a) == 0 or len(b) == 0:
        return 1.0 - f1score
    buff1_a = {}
    buff2_a = {}
    buff1_b = {}
    buff2_b = {}
    if lang == 'eng':
        buff1_a, buff2_a = ExtractHprnmsEng(a)
#         print('buff_a ', buff1_a, 'buff2_a ', buff2_a)
        buff1_b, buff2_b = ExtractHprnmsEng(b)
#         print('')
#         print('len(buff_a) ', len(buff_a))
#         print('len(buff_b) ', len(buff_b))
    if len(buff1_a.intersection(buff1_b)) > 0:
#         print('len(buff_a.intersection(buff_b))', len(buff_a.intersection(buff_b)), '\n')
        overlap_hyp_cnt += 1
#         print('overlap_hyp_cnt ', overlap_hyp_cnt, '\n')
    if len(buff2_a.intersection(buff2_b)) > 0:
        overlap_hyp_cnt += 0.5
    recall = overlap_hyp_cnt/len(a)
#     print('recall', recall)
    precision = overlap_hyp_cnt/len(b)
#     print('precision', precision, '\n')
    f1score_hyp = 2*recall/(recall + precision) if overlap_hyp_cnt > 0 else 0
    f1res = (2*f1score+f1score_hyp)/3
    return (1.0 - f1res)

In [10]:
buff = list(res.items())
dist = np.zeros((len(buff),len(buff)))
for lli,ll in enumerate(buff):
    for rri,rr in enumerate(buff):
        dist[lli,rri] = distance(ll[0],rr[0])
#         print('(ll[0],rr[0]) ', (ll[0],rr[0]), '\n')
dist

array([[0.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 0.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 0.80952381, 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 0.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 0.62962963,
        1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 0.        , 0.75757576,
        1.        , 1.        , 0.85964912, 1.        , 1.        ,
        1.        , 1.        , 1.        , 1

#### Top ten closest articles with fuzzy metrics of titles

In [11]:
@interact(ind=(0,len(buff)-1,1))
def h(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint([buff[i][0] for i in dist[ind][:].argsort()[1:11]])

interactive(children=(IntSlider(value=0, description='ind', max=19), Output()), _dom_classes=('widget-interact…

In [12]:
@interact(ind=(0,len(buff)-1,1))
def hypernyms(ind=0):
    pp = pprint.PrettyPrinter(indent=4)
    print(' '.join(buff[ind][0]))
    pp.pprint(buff[ind][1])

interactive(children=(IntSlider(value=0, description='ind', max=19), Output()), _dom_classes=('widget-interact…